## 长短时记忆

### 1 LSTM网络结构
LSTM 中引入了3个门，即输入门（input gate）、遗忘门（forget gate）和输出门（output gate），以及与隐藏状态形状相同的记忆细胞（某些文献把记忆细胞当成一种特殊的隐藏状态），从而记录额外的信息。

与GRU一样，LSTM的遗忘门、输入门和输出门的输入均为当前时间步的输入$X_t$和上一时间步的隐藏状态$H_{t-1}$，输出由激活函数为sigmoid函数的全连接层计算得到。三个门的输出均在0到1之间。计算表达式如下：

$$
I_t = \sigma (X_t W_{xi} + H_{t-1} W_{hi} + \vec{b}_i) \in \mathbb{R}^{n \times h}\\
F_t = \sigma (X_t W_{xf} + H_{t-1} W_{hf} + \vec{b}_f) \in \mathbb{R}^{n \times h}\\
O_t = \sigma (X_t W_{xo} + H_{t-1} W_{ho} + \vec{b}_o) \in \mathbb{R}^{n \times h},
$$

其中$W_{xi},W_{xf},W_{xo} \in \mathbb{R}^{d \times h}$和$W_{hi},W_{hf},W_{ho} \in \mathbb{R}^{h \times h}$为权重参数，$\vec{b}_i, \vec{b}_f, \vec{b}_o \in \mathbb{R}^{1 \times h}$为偏置。

![avatar](../resource/lstm_0.svg)

**计算候选记忆细胞$\tilde{C}_t \in \mathbb{R}^{n \times h}$**。采用tanh作为激活函数，有

$$
\tilde{C}_t = tanh(X_t W_{xc} + H_{t-1}W_{hc} + \vec{b}_c) \in \mathbb{R}^{n \times h},
$$
其中$W_{xc} \in \mathbb{R}^{d \times h}$和$W_{hc} \in \mathbb{R}^{h \times h}$为权重参数，$\vec{b}_c \in \mathbb{R}^{1 \times h}$为偏置。

![avatar](../resource/lstm_1.svg)

**计算记忆细胞$C_t \in \mathbb{R}^{n \times h}$**。我们可以通过元素值域在$[0,1]$的输入门、遗忘门和输出门来控制隐藏状态中信息的流动，这一般也是通过使用按元素乘法来实现的。当前时间步记忆细胞$C_t$的计算组合了上一时间步记忆细胞和当前时间步候选记忆细胞的信息，并通过遗忘门和输入门来控制信息的流动：

$$
C_t = F_t \odot C_{t-1} + I_t \odot \tilde{C}_t.
$$

遗忘门控制上一时间步的记忆细胞$C_{t-1}$中的信息是否传递到当前时间步，而输入门则通过候选记忆细胞$\tilde{C}_t$控制当前时间步的输入$X_t$如何流入当前时间步的记忆细胞。如果遗忘门一直近似1且输入门一直近似0，过去的记忆细胞将一直通过时间保存并传递至当前时间步。这个设计可以应对循环神经网络中的梯度衰减问题，并更好地捕捉时间序列中时间步距离较大的依赖关系。

![avatar](../resource/lstm_2.svg)

**计算隐藏状态$H_t \in \mathbb{R}^{n \times h}$**。输出门来控制从记忆细胞到隐藏状态$H_t$的信息的流动：

$$
H_t = O_t \odot tanh(C_t).
$$

这里的tanh函数确保隐藏状态元素值在-1到1之间。需要注意的是，当输出门近似1时，记忆细胞信息将传递到隐藏状态供输出层使用；当输出门近似0时，记忆细胞信息只自己保留。

### 2 从零开始实现

In [2]:
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F
import my_utils

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

corpus_indices, char_to_idx, idx_to_char, vocab_size = my_utils.load_data_jay_lyrics()

In [3]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return nn.Parameter(ts, requires_grad=True)
    def _three():
        return (_one((num_inputs, num_hiddens)),
                _one((num_hiddens, num_hiddens)),
                torch.nn.Parameter(torch.zeros(num_hiddens, device=device, dtype=torch.float32), requires_grad=True))
    
    W_xi, W_hi, b_i = _three()  # 输入门参数
    W_xf, W_hf, b_f = _three()  # 遗忘门参数
    W_xo, W_ho, b_o = _three()  # 输出门参数
    W_xc, W_hc, b_c = _three()  # 候选记忆细胞参数

    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, dtype=torch.float32), requires_grad=True)
    return nn.ParameterList([W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q])

In [4]:
def init_lstm_state(batch_size, num_hiddens, device):
    # 返回额外的形状为(批量大小, 隐藏单元个数)的以藏状态和值为0的记忆细胞
    return (torch.zeros((batch_size, num_hiddens), device=device), 
            torch.zeros((batch_size, num_hiddens), device=device))

In [5]:
def lstm(inputs, state, params):
    [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q] = params
    (H, C) = state
    outputs = []
    for X in inputs:
        I = torch.sigmoid(torch.matmul(X, W_xi) + torch.matmul(H, W_hi) + b_i)
        F = torch.sigmoid(torch.matmul(X, W_xf) + torch.matmul(H, W_hf) + b_f)
        O = torch.sigmoid(torch.matmul(X, W_xo) + torch.matmul(H, W_ho) + b_o)
        C_tilda = torch.tanh(torch.matmul(X, W_xc) + torch.matmul(H, W_hc) + b_c)
        C = F * C + I * C_tilda
        H = O * C.tanh()
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H, C)

In [6]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

In [7]:
my_utils.rnn_train_and_predict(lstm, get_params, init_lstm_state, num_hiddens,
                               vocab_size, device, corpus_indices, idx_to_char,
                               char_to_idx, False, num_epochs, num_steps,
                               lr, clipping_theta, batch_size, pred_period,
                               pred_len, prefixes)

epoch 40, perplexity 210.808170, time 1.23 sec
 - 分开 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我
 - 不分开 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我
epoch 80, perplexity 65.713858, time 1.18 sec
 - 分开 我想你这你 我不要这样 我不要这 我不要这 我不要这 我不要这 我不要这 我不要这 我不要这 我不
 - 不分开 我想你这你 我不要这 我不要这 我不要这 我不要这 我不要这 我不要这 我不要这 我不要这 我不要
epoch 120, perplexity 15.067958, time 1.18 sec
 - 分开 我想你 你你我 想你是这对我 有你去觉 我该了这生奏 后知后觉 我想好这生活 我知好觉 我该了好节
 - 不分开 我想你 你你我 我想就这样着我 别知抖觉 我该了这生活 我知好觉 我该了这节奏 后知后觉 我想好好
epoch 160, perplexity 3.883025, time 1.34 sec
 - 分开 我想带你你经着  想想你的生笑 我想想你的微笑 我想想你的微笑 我想想你的微笑 我想想你的微笑 我
 - 不分开 我已这你生活 我不想觉生活 不知不觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋


### 3 简洁实现

In [8]:
lr = 1e-2
lstm_layer = nn.LSTM(input_size=vocab_size, hidden_size=num_hiddens)
model = my_utils.RNNModel(lstm_layer, vocab_size).to(device)
my_utils.rnn_train_and_predict_torch(model, num_hiddens, vocab_size, device, 
                                     corpus_indices, idx_to_char, char_to_idx, 
                                     num_epochs, num_steps, lr, clipping_theta, 
                                     batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 1.021912, time 0.84 sec
 - 分开始移动 回到当初爱你的时空 停格内容不忠 所有回忆对着我进攻 我的伤口被你拆封 誓言太沉重泪被纵容 
 - 不分开始移动 回到当初爱你的时空 停格内容不忠 所有回忆对着我进攻 我的伤口被你拆封 誓言太沉重泪被纵容 
epoch 80, perplexity 1.029642, time 0.82 sec
 - 分开始移动 回到当初爱你的时空 停格内容不忠 所有回忆对着我进攻       所有回忆 我的一口吴侬软语
 - 不分开始移动 回到当初爱你的时空 停格内容不忠 所有回忆对着我进攻       所有回忆 我的一口吴侬软语
epoch 120, perplexity 1.009221, time 0.91 sec
 - 分开始移动 回到当初爱你的时空 停格内容不忠 所有回忆对着我进攻 我的伤口被你拆封 誓言太沉重泪被纵容 
 - 不分开始移动 回到当初爱你的时空 停格内容不忠 所有回忆对着我进攻 我的伤口被你拆封 誓言太沉重泪被纵容 
epoch 160, perplexity 1.012794, time 0.82 sec
 - 分开始移动 回到当初爱你的时空 停格内容不忠 所有回忆对着我进攻 我的伤口被你拆封 誓言太沉重泪被纵容 
 - 不分开始想要去河口 默默的在等著我 家乡的爹娘早已苍老了轮廓 娘子我欠你太多 一壶好酒 再来一碗热粥 配上
